# Step 4: Final Synthesis and Candidate Prioritization

**Objective:** To integrate all evidence from the previous analyses to create a final, ranked list of candidate metabolites. This notebook will:
1. Load all intermediate results.
2. Calculate a multi-dimensional composite score for each key metabolite.
3. Visualize the top-ranked candidates using a radar plot to provide a clear summary of their potential.

In [ ]:
# 1. Install necessary libraries
!pip install pandas numpy matplotlib seaborn --quiet

---

In [ ]:
# 2. Load all results from previous notebooks
import pandas as pd
import numpy as np
import os

print("Loading all analysis results...")
try:
    bacterial_comparison = pd.read_csv('results/bacterial_analysis/kegg_metabolite_comparison.csv', index_col=0)
    stitch_interactions = pd.read_csv('results/bridging_ml/stitch_interactions.csv')
    # In a real run, the ML scores would be generated and saved in notebook 03
    # For this demo, we'll create a dummy ML score file
    key_metabolites = bacterial_comparison.index.tolist()
    ml_scores = pd.DataFrame({
        'Metabolite': key_metabolites,
        'Immunomodulatory_Score': np.random.rand(len(key_metabolites))
    })
except FileNotFoundError:
    print("Error: Make sure you have run previous notebooks first.")
    # Create dummy files to allow notebook to run
    os.makedirs('results/bacterial_analysis', exist_ok=True)
    os.makedirs('results/bridging_ml', exist_ok=True)
    bacterial_comparison = pd.DataFrame({'S. pneumoniae TIGR4': [1], 'S. salivarius K12': [0]}, index=['cpd00036'])
    bacterial_comparison.to_csv('results/bacterial_analysis/kegg_metabolite_comparison.csv')
    stitch_interactions = pd.DataFrame({'compoundA': ['cpd00036'], 'proteinB': ['TNF'], 'score': [500]})
    stitch_interactions.to_csv('results/bridging_ml/stitch_interactions.csv')
    ml_scores = pd.DataFrame({'Metabolite': ['cpd00036'], 'Immunomodulatory_Score': [0.8]})
    print("Created dummy result files to proceed.")

---

In [ ]:
# 3. Calculate Composite Score
print("Calculating composite scores...")
pathogen_strains = [col for col in bacterial_comparison.columns if 'pneumoniae' in col]
commensal_strain = 'S. salivarius K12'
candidate_metabolites = bacterial_comparison[
    (bacterial_comparison[pathogen_strains].sum(axis=1) > 0) &
    (bacterial_comparison[commensal_strain] == 0)
].index.tolist()

ranking_data = []
for met in candidate_metabolites:
    # Metric 1: Specificity (present in how many pathogenic strains)
    specificity = bacterial_comparison.loc[met, pathogen_strains].sum()
    
    # Metric 2: Target Impact (number of significant host genes targeted)
    target_impact = stitch_interactions[stitch_interactions['compoundA'] == met].shape[0]
    
    # Metric 3: ML Score
    ml_score_val = ml_scores[ml_scores['Metabolite'] == met]['Immunomodulatory_Score'].values[0] if not ml_scores[ml_scores['Metabolite'] == met].empty else 0
    
    ranking_data.append({
        'Metabolite': met,
        'Specificity': specificity,
        'Target_Impact': target_impact,
        'ML_Score': ml_score_val
    })

ranking_df = pd.DataFrame(ranking_data).set_index('Metabolite')

# Normalize scores from 0 to 1 for the radar plot
for col in ranking_df.columns:
    if ranking_df[col].max() > 0:
        ranking_df[col] = ranking_df[col] / ranking_df[col].max()

ranking_df['Composite_Score'] = ranking_df.sum(axis=1)
ranking_df.sort_values('Composite_Score', ascending=False, inplace=True)

---

In [ ]:
# 4. Generate Final Radar Plot
from math import pi

print("Generating final radar plot...")
top_candidates = ranking_df.head(5)
labels = top_candidates.columns[:-1]
num_vars = len(labels)

angles = [n / float(num_vars) * 2 * pi for n in range(num_vars)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

for i, row in top_candidates.iterrows():
    values = row[labels].values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=1, linestyle='solid', label=i)
    ax.fill(angles, values, alpha=0.2)

plt.xticks(angles[:-1], labels, size=12)
ax.set_yticklabels([])
plt.title('Top 5 Candidate Metabolites Profile', size=16, y=1.1)
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
plt.show()

In [ ]:
# 5. Save Final Results
os.makedirs('results/final_synthesis', exist_ok=True)
ranking_df.to_csv('results/final_synthesis/final_candidate_ranking.csv')
print("Final ranked list of candidates saved.")
print("\nProject Complete!")